In [1]:


import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from statsmodels.api import OLS, add_constant
import pandas_datareader.data as web

from linearmodels.asset_pricing import LinearFactorModel

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')



In [2]:
ff_factor = 'F-F_Research_Data_5_Factors_2x3'
ff_factor_data = web.DataReader(ff_factor, 'famafrench', start='2010', end='2017-12')[0]
ff_factor_data.info()


<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 96 entries, 2010-01 to 2017-12
Freq: M
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  96 non-null     float64
 1   SMB     96 non-null     float64
 2   HML     96 non-null     float64
 3   RMW     96 non-null     float64
 4   CMA     96 non-null     float64
 5   RF      96 non-null     float64
dtypes: float64(6)
memory usage: 5.2 KB


In [3]:
ff_factor_data.describe()

,Mkt-RF,SMB,HML,RMW,CMA,RF
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,1.158438,0.060417,-0.049792,0.128750,0.047917,0.012604
std,3.580012,2.302008,2.202941,1.581701,1.411604,0.022583
min,-7.890000,-4.550000,-4.700000,-3.880000,-3.250000,0.000000
25%,-0.917500,-1.695000,-1.662500,-1.090000,-0.952500,0.000000
50%,1.235000,0.205000,-0.310000,0.190000,-0.005000,0.000000
75%,3.197500,1.590000,1.205000,1.197500,0.922500,0.010000
max,11.350000,7.070000,8.210000,3.480000,3.700000,0.090000


In [4]:
ff_portfolio = '17_Industry_Portfolios'
ff_portfolio_data = web.DataReader(ff_portfolio, 'famafrench', start='2010', end='2017-12')[0]
ff_portfolio_data = ff_portfolio_data.sub(ff_factor_data.RF, axis=0)
ff_portfolio_data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 96 entries, 2010-01 to 2017-12
Freq: M
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Food    96 non-null     float64
 1   Mines   96 non-null     float64
 2   Oil     96 non-null     float64
 3   Clths   96 non-null     float64
 4   Durbl   96 non-null     float64
 5   Chems   96 non-null     float64
 6   Cnsum   96 non-null     float64
 7   Cnstr   96 non-null     float64
 8   Steel   96 non-null     float64
 9   FabPr   96 non-null     float64
 10  Machn   96 non-null     float64
 11  Cars    96 non-null     float64
 12  Trans   96 non-null     float64
 13  Utils   96 non-null     float64
 14  Rtail   96 non-null     float64
 15  Finan   96 non-null     float64
 16  Other   96 non-null     float64
dtypes: float64(17)
memory usage: 13.5 KB


In [5]:
ff_portfolio_data.describe()

,Food,Mines,Oil,Clths,Durbl,Chems,Cnsum,Cnstr,Steel,FabPr,Machn,Cars,Trans,Utils,Rtail,Finan,Other
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,1.035417,0.210000,0.594896,1.383646,1.141771,1.327604,1.189063,1.734167,0.535521,1.350208,1.219271,1.282292,1.469792,0.894271,1.240417,1.247292,1.286250
std,2.805824,7.942856,5.504306,5.007997,5.207729,5.585949,3.141205,5.249994,7.424776,4.713899,4.801381,5.757512,4.167801,3.232446,3.519087,4.868399,3.674353
min,-5.160000,-24.510000,-11.760000,-9.770000,-13.170000,-16.770000,-7.180000,-14.140000,-20.570000,-12.180000,-9.110000,-11.770000,-8.660000,-7.110000,-9.200000,-11.290000,-8.040000
25%,-0.797500,-5.810000,-3.167500,-1.572500,-2.272500,-1.337500,-0.895000,-2.435000,-4.385000,-1.450000,-2.010000,-1.255000,-0.860000,-0.735000,-0.952500,-1.530000,-1.147500
50%,0.940000,-0.480000,0.990000,1.155000,1.225000,1.445000,1.465000,2.160000,0.655000,1.500000,1.495000,0.620000,1.520000,1.275000,0.885000,1.790000,1.570000
75%,3.210000,5.905000,4.195000,3.770000,4.627500,4.437500,3.295000,5.555000,4.270000,3.845000,4.582500,4.815000,4.342500,2.930000,3.367500,4.147500,3.425000
max,6.680000,22.030000,16.030000,17.320000,16.580000,18.360000,8.280000,15.580000,21.620000,17.870000,14.740000,21.040000,13.030000,7.840000,12.460000,13.500000,10.430000


In [6]:


with pd.HDFStore('./data/assets.h5') as store:
    prices = store['/quandl/wiki/prices'].adj_close.unstack().loc['2010':'2017']
    equities = store['/us_equities/stocks'].drop_duplicates()




KeyError: 'No object named /us_equities/stocks in the file'